In [ ]:
import json
import random
import itertools
import sys
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import category_encoders

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers
from datetime import datetime

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# check that correct virtual env is being used
#print(sys.executable)
print(sys.version)

# list all installed modules
#%pip list

In [ ]:
def number_of_unique_values_per_column(dataframe: pd.DataFrame) -> None:
    print([(column, dataframe[column].nunique()) for column in dataframe.columns])

# remove all columns that don't contain any important information, since all entries have the same entry
def remove_columns_with_a_single_value(original_dataframe: pd.DataFrame) -> pd.DataFrame:
    return original_dataframe[[column for column in all_sessions.columns if len(all_sessions[column].unique()) > 1]]

def move_column_to_the_front(original_dataframe: pd.DataFrame, column_to_move: str) -> pd.DataFrame:
    first_column = original_dataframe.pop(column_to_move)
    original_dataframe.insert(0, column_to_move, first_column)
    return original_dataframe

def encode_time(original_dataframe: pd.DataFrame, columns_to_encode: list[str]) -> pd.DataFrame:
    new_df = pd.DataFrame()
    
    for index in range(0, len(columns_to_encode)):
        original_dataframe[columns_to_encode[index]] = original_dataframe[columns_to_encode[index]].astype('float64')
        new_df[columns_to_encode[index]] = np.sin(2 * np.pi * original_dataframe[columns_to_encode[index]] / 24.0)
    original_dataframe.update(new_df)

    return original_dataframe

def encode_utc(original_dataframe: pd.DataFrame, columns_to_encode: list[str]) -> pd.DataFrame:
    new_df = pd.DataFrame()
    pattern = "%Y-%m-%dT%H:%M:%S.%fZ"

    for index in range(0, len(columns_to_encode)):
        if index == 0: # time_utc
            pattern = "%Y-%m-%dT%H:%M:%S.%fZ"
        else: # time_local
            pattern = "%Y-%m-%dT%H:%M:%S.%f"

        timestamps = []
        for i in range(len(original_dataframe)):
            time_stamp = datetime.strptime(original_dataframe.loc[i, columns_to_encode[index]], pattern)
            timestamps.append(time_stamp.timestamp())
        new_df[columns_to_encode[index]] = timestamps
    
    original_dataframe.update(new_df)
    
    return original_dataframe

def binary_encode_ids(original_dataframe: pd.DataFrame, columns_to_encode: list[str]) -> pd.DataFrame:
    binary_encoder = category_encoders.BinaryEncoder(cols=columns_to_encode, return_df=True)
    original_dataframe = binary_encoder.fit_transform(original_dataframe)
    return original_dataframe

def encode_boolean(original_dataframe: pd.DataFrame, columns_to_encode: list[str]) -> pd.DataFrame:
    for index in range(0, len(columns_to_encode)):
        original_dataframe[columns_to_encode[index]] = original_dataframe[columns_to_encode[index]].astype('int')
    return original_dataframe

def encode_classification(original_dataframe: pd.DataFrame, column_to_encode: str) -> pd.DataFrame:
    label_encoder = LabelEncoder()
    original_dataframe[column_to_encode] = label_encoder.fit_transform(all_sessions[column_to_encode])
    return original_dataframe

def get_weather_data(original_dataframe: pd.DataFrame, column_to_encode: str, json_subarray_name: str) -> pd.DataFrame:
    weather_data_list = []
    
    for _, row in original_dataframe.iterrows():
        weather_day_id = row[column_to_encode]
        weather_day_data = parsed_json[json_subarray_name][str(weather_day_id)]
        weather_day_data = {k: v for k, v in weather_day_data.items() if k != 'id'}
        weather_data_list.append(weather_day_data)
    
    weather_data_df = pd.DataFrame(weather_data_list)
    
    original_dataframe = pd.concat([original_dataframe, weather_data_df], axis=1)
    
    return original_dataframe

# read json data
with open("datasets\\transfer\\smaller_dataset.json") as file:
    parsed_json = json.load(file)

# build dataframe 
total_amount_of_rows = 0
all_sessions = pd.DataFrame()
for i in pd.json_normalize(parsed_json['traces']):
    total_amount_of_rows += len(pd.json_normalize(parsed_json['traces'][i]))
    single_session = pd.json_normalize(parsed_json['traces'][i])
    all_sessions = pd.concat([all_sessions, single_session], ignore_index=True)

all_sessions = remove_columns_with_a_single_value(all_sessions)

# encode 24h time format
all_sessions = encode_time(all_sessions, ['time_dow', 'time_hod'])

# encode UTC Timestamp
all_sessions = encode_utc(all_sessions, ['time_utc', 'time_local'])

# encode boolean values to int values
all_sessions = encode_boolean(all_sessions, ['device_online'])

# binary encode session_id and device_id
all_sessions = binary_encode_ids(all_sessions, ['session_id', 'device_id'])

# encode the classifcation column 
all_sessions = encode_classification(all_sessions, 'content_id')

# move classification column to the front 
all_sessions = move_column_to_the_front(all_sessions, 'content_id')

all_sessions = get_weather_data(all_sessions, 'weather_day_id', 'weather_day_map')
all_sessions = get_weather_data(all_sessions, 'weather_hour_id', 'weather_hour_map')
print(len(all_sessions.columns))

# grouped all entries by session_id
grouped = all_sessions.groupby(['session_id_0', 'session_id_1', 'session_id_2', 'session_id_3', 'session_id_4', 'session_id_5', 'session_id_6'])

# number of columns 
number_of_columns = all_sessions.shape[1]

feature_vectors = []
for name, group in grouped:
    iterator, classification_iterator = itertools.tee(group.itertuples(index=False))
    next(classification_iterator, None) 
    
    prev_prev_feature_vector = ['UNKNOWN'] * number_of_columns
    prev_feature_vector = ['UNKNOWN'] * number_of_columns
    feature_vector = []
    vector_to_add = []
    
    for row in iterator:
        # geginning with the second iteration move all feature_vectors back one step
        if prev_feature_vector != []:
            prev_prev_feature_vector = prev_feature_vector
        
        if feature_vector != []:
            prev_feature_vector = feature_vector

        # build current_feature_vector
        feature_vector = [str(element) for element in row]

        # get the classifier of the next event for the feature_vector   
        classification = 'UNKNOWN'
        try:
            next_row = next(classification_iterator)
            classification = next_row[0] 
        except StopIteration:
            classification = 'UNKNOWN'

        # build feature_vector with prev_prev and prev_feature_vector
        vector_to_add = [prev_prev_feature_vector, prev_feature_vector, feature_vector]

        # append feature_vector to all feature_vectors
        feature_vectors.append((vector_to_add, classification))

# shuffle the dataset 
random.shuffle(feature_vectors)


''' 
for i in reversed(range(len(feature_vectors))):
    feature = feature_vectors[i]
    if 'UNKNOWN' in feature[0][0] or 'UNKNOWN' in feature[0][1] or 'UNKNOWN' in feature[0][2] or isinstance(feature[1], str):
        del feature_vectors[i]
    elif 'None' in feature[0][0] or 'None' in feature[0][1] or 'None' in feature[0][2]:
        del feature_vectors[i]    


for i in range(len(feature_vectors)):
    feature_vector = feature_vectors[i][0]
    classification = feature_vectors[i][1]
    feature_vector = [[float(element) for element in sublist] for sublist in feature_vector]
    feature_vectors[i] = (feature_vector, classification)

data = [([input_features], label) for input_features, label in feature_vectors]

input_features = [features for features, _ in data]
labels = [label for _, label in data]
concatened_input_features = np.concatenate(input_features)


X_train, X_test, y_train, y_test = train_test_split(input_features, labels, test_size=0.2, random_state=42)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)



#print dataframe row by row
for i in range(len(all_sessions)):
    print(all_sessions.loc[i])
    break
'''